<a href="https://colab.research.google.com/github/dernameistegal/airbnb_price/blob/main/data_utils/munich/translate_reviews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Translation of airbnb dataset into english

In [1]:
from google.colab import drive
import pandas as pd
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Preliminary works
- add comments_en column with values None
- add lang column with values None
- replace na values in comments colum with "no text"

In [8]:
%%capture
!pip install git+https://github.com/neuml/txtai#egg=txtai[pipeline]

In [2]:
import torch
path = "/content/drive/MyDrive/Colab/airbnb/munich/reviews_workfile.pickle"
reviews = pd.read_pickle(path)

In [10]:
import torch
path = "/content/drive/MyDrive/Colab/airbnb/munich/listings.pickle"
listings = pd.read_pickle(path)

In [17]:
reviews.listing_id.isin(listings.id).all()

True

In [18]:
from torch.utils.data import DataLoader, Dataset

class Reviews(Dataset):
    def __init__(self, reviews):
        super().__init__()
        self.reviews = reviews

    def __getitem__(self, key):
        return self.reviews["comments"].iloc[key], self.reviews.index[key]

    def __len__(self):
        return len(self.reviews)

In [19]:
dataset = Reviews(reviews)
dataloader = DataLoader(dataset, batch_size=200)

# Detect language and sort by it

In [21]:
# detect language
from txtai.pipeline import Translation
translate = Translation(batch=200)
with torch.no_grad():
    for i, (text, id) in enumerate(dataloader):
        text = list(text)
        try: 
            if reviews.loc[id, "lang"].isnull().values.any():
                language = translate.detect(list(text))
                reviews.loc[id, "lang"] = language
        except:
          continue
        last_id = id[-1]
        print(i)

Downloading:   0%|          | 0.00/916k [00:00<?, ?B/s]

0
1
2
3
4
5
8
9
10
11
12
13
14
15
16
17
18
19
20
21
23
24
25
26
27
28
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
72
73
74
75
78
79
80
81
82
84
85
86
88
90
91
94


In [22]:
reviews = reviews.sort_values(by="lang")

In [32]:
# remove text garbage	
reviews["comments_en"] = reviews["comments_en"].str.replace("<.*?>", "")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  


In [33]:
reviews["comments_en"].iloc[0]

"My stay at Ralph's place was amazing. Place is great and has everything a Traveler would need. Ralph was a great host and always up to help if needed. Place is spacious and close to public transportation.br/>Raff apartment is wonderful and comfortable. Everything the traveller needs. Ralph was an excellent host."

In [34]:
path = "/content/drive/MyDrive/Colab/airbnb/munich/reviews_workfile.pickle"
reviews.to_pickle(path)

# Translate text

In [24]:
len(reviews) / 64

298.796875

In [ ]:
!python script.py

0 already done
1
Token indices sequence length is longer than the specified maximum sequence length for this model (782 > 512). Running this sequence through the model will result in indexing errors


In [35]:

#@title this is script.py
from txtai.pipeline import Translation
import torch
from torch.utils.data import DataLoader, Dataset
import pandas as pd

# Create translation model
translate = Translation(batch=50)

path = "/content/drive/MyDrive/Colab/airbnb/munich/reviews_workfile.pickle"
reviews = pd.read_pickle(path)

class Reviews(Dataset):
    def __init__(self, reviews):
        super().__init__()
        self.reviews = reviews

    def __getitem__(self, key):
        return self.reviews["comments"].iloc[key], self.reviews.index[key]

    def __len__(self):
        return len(self.reviews)

dataset = Reviews(reviews)
dataloader = DataLoader(dataset, batch_size=50)


with torch.no_grad():
    for i, (text, id) in enumerate(dataloader):
        text = list(text)
        if reviews.loc[id, "comments_en"].isnull().values.any():

            sourcefinder = reviews.loc[id, "lang"].unique()
            if len(sourcefinder) > 1:
                  print(sourcefinder)
                  sourcefinder = None
            else:
                sourcefinder = sourcefinder[0]
                print(i)

            translation = translate(list(text), "en", source = sourcefinder)
            reviews.loc[id, "comments_en"] = translation
            if i % 50 == 0:
                path = "/content/drive/MyDrive/Colab/airbnb/munich/reviews_workfile.pickle"
                reviews.to_pickle(path)
        else:
            print(i, "already done")



0 already done
1


Token indices sequence length is longer than the specified maximum sequence length for this model (782 > 512). Running this sequence through the model will result in indexing errors


RuntimeError: ignored

In [ ]:
path = "/content/reviews_save"
reviews = pd.read_pickle(path)

In [ ]:
path = "/content/drive/MyDrive/Colab/airbnb/data/translations/translated_reviews.pickle"
reviews.to_pickle(path)

In [ ]:
path = "/content/drive/MyDrive/Colab/airbnb/data/translations/listings"
listings = pd.read_pickle(path)

In [ ]:
path = "/content/drive/MyDrive/Colab/airbnb/data/translations/translated_listings.pickle"
listings.to_pickle(path)